In [ ]:
import socket
import pymysql
import time
import datetime
from dateutil.parser import parser
import math
import sys

def total_time(date,device_num):
        print("total time start...")
        global time1
        global time2
        if(time1 == 0 and time2 == 0):
                time1 = date
#               a=date
##              print(time1)
#               start = datetime.datetime.strptime(a,'%Y-%m-%d %H:%M:%S')
                print("start time init...")
        elif(time1 != 0 and time2 == 0):
                time2 = date
#               print(time1,time2)
#               current = datetime.datetime.strptime(time2,'%Y-%m-%d %H:%M:%S') # Current date data
#               print(start,current)
#               using_time = (current - start)                                  # Calcurate total using time
                current = (datetime.datetime.strptime(time2, '%Y-%m-%d %H:%M:%S'))
                start = (datetime.datetime.strptime(time1, '%Y-%m-%d %H:%M:%S'))
                using_time = (current-start)
#               print("Using time : ",using_time)
                time1 = date
                time2 = 0
                sql='SELECT * FROM Device_info WHERE Device_number="%s"'%(device_num)
                cur.execute(sql)
                result = cur.fetchone()
                result = str(result[3])
                if(len(result)<=8):
                        result = result.split(":")
                        hour = int(result[0])*3600
                        min = int(result[1])*60
                        sec = int(result[2])
                        hour = hour+min+sec
                        before_usingtime = datetime.timedelta(0,hour)
                        hour = 0
#                       print(before_usingtime)
                elif(len(result)>8):
                        result = str(result).split("u")
                        result = result[4].split("'")
                        tmp = result[1].split(' ')
                        day = int(tmp[0])
                        tmp = tmp[2].split(':')
                        hour = int(tmp[0])*3600
                        min = int(tmp[1])*60
                        sec = int(tmp[2])
                        hour = int(tmp[2])+hour+min
                        before_usingtime = datetime.timedelta(day,hour)                 # type transform
                        hour = 0
                        #                       print(before_usingtime)
                total_usingtime = using_time + before_usingtime
                using_time = 0
                print(total_usingtime)
                sql = 'UPDATE Device_info SET Total_Usingtime="%s" WHERE Device_number="%s"' % (total_usingtime,devi$
                cur.execute(sql)
                db.commit()
                print("total time complete...")

def harversine(gps1_lat,gps1_lon,gps2_lat,gps2_lon,num):
        print("harversine start...")
        R = 6371
        lat1 = float(gps1_lat)
        lon1 = float(gps1_lon)
        lat2 = float(gps2_lat)
        lon2 = float(gps2_lon)
        eq_device_num = num

        phi1, phi2 = math.radians(lat1), math.radians(lat2)
        dphi       = math.radians(lat2 - lat1)
        dlambda    = math.radians(lon2 - lon1)

        a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
        result = 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))
#       print(result)
        today_distance = result
        sql = 'select * from Device_info where Device_number="%s"'%(num)
        cur.execute(sql)
        total_distance = cur.fetchone()
        #total_distance = str(total_distance).split("u")
        total_distance = total_distance[4].split("'")
        total_distance = float(total_distance[0])
        Final_total_distance = (total_distance + today_distance)                        #Final distance
        print(Final_total_distance)
        sql = 'UPDATE Device_info SET Total_distance="%s" WHERE Device_number="%s"' % (Final_total_distance,eq_devic$
        cur.execute(sql)
        db.commit
        print("harversine complete...")

def GPS_set_and_distance(gps1_lat,gps1_lon,num):
        print("GPS_set_and_distance start...")
        global GPS_lat1
        global GPS_lon1
        global GPS_lat2
        global GPS_lon2
        eq_device_num = num
        if(GPS_lat1 == 0 and GPS_lon1 == 0):
                GPS_lat1 = gps1_lat
                GPS_lon1 = gps1_lon
#               print("GPS_lat1 : ",GPS_lat1)
#               print("GPS_lon1 : ",GPS_lon1)
        elif(GPS_lat1 != 0 and GPS_lon1 != 0 and GPS_lat2 == 0 and GPS_lon2 == 0):
                GPS_lat2 = gps1_lat
                GPS_lon2 = gps1_lon
#               print(eq_device_num)
                harversine(GPS_lat1,GPS_lon1,GPS_lat2,GPS_lon2,eq_device_num)
                print("GPS_position replace...")
                GPS_lat1 = GPS_lat2
                GPS_lon1 = GPS_lon2
                GPS_lat2 = 0
                GPS_lon2 = 0
        print("GPS_set_and_distance complete...")

def on_message(message):
        print("on_message start...")
        now=time.localtime()
        try:
                temp_time= "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now$
                long = "1" + message.encode("hex")[0:2]+ "." + message.encode("hex")[2:8]
                lat = message.encode("hex")[8:10] +"." + message.encode("hex")[10:16]
                equip_number = message.encode("hex")[16:18]
                temperature = message.encode("hex")[18:22]
                humidity = message.encode("hex")[22:26]
                door = message.encode("hex")[26:27]
                cooling = message.encode("hex")[27:28]
        except:
                pass
#chk_data start
        chk_data(temp_time,equip_number,long,lat)

        if(equip_number == '9'):
                cur.excute(sql2,(temp_time,equip_number,long,lat))
                db.commit()
                cur.excute(sql3,(temp_time,equip_number,temperature,humidity,door,cooling))
                db.commit()
        else:
                cur.excute(sql,(temp_time,equip_number,long,lat))
                db.commit()
def chk_data(temp_time,equip_number,long,lat):
        global chk_flag
        sql='SELECT * FROM state_GPS_data ORDER BY time_t DESC LIMIT 1'
        print("chk_data start...")
        m1 = (temp_time,equip_number,long,lat)
        cur.execute(sql)
        result = cur.fetchall()
#       print(result)
        if(m1==result):
                chk_flag=1 #chk_flag=1 main_loop init
                print("warning!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("chk_data complete...")

chk_flag=0
GPS_lat1 = 0
GPS_lon1 = 0
GPS_lat2 = 0
GPS_lon2 = 0
time1 = 0
time2 = 0
db=pymysql.connect(host='127.0.0.1',port=3306,user='VSS',passwd='password',db='db',charset='utf8')
cur=db.cursor()
sql1='INSERT INTO `state_GPS_data`(`time_t`, `equip_number`, `longitude`, `latitude`) VALUES(%s,%s,%s,%s)'
sql2='INSERT INTO `state_CTgps_data`(`time_t`, `equip_number`, `longitude`, `latitude`) VALUES(%s,%s,%s,%s)'
sql3='INSERT INTO `Container_info_env`(`time_t`,`CT_info_env_id`,`Temperature`,`Humidity`,`Door`,`Cooling`) VALUE(%s$
#sql4='INSERT INTO `Device_info`(`Device_number`,`Check_data`,`Check_Reason`,`Total_Usingtime`,`Total_distance`) VAL$
sock=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
ip='0.0.0.0'
port=5959
#server
sock.bind((ip,port))
print("listen:",port)

while 1 :
        chk_flag=0
##data receive
        data,addr=sock.recvfrom(150)
        data = data.decode()
#       print("data : ",data.decode())
        num = data.split('/')
        print("current Device num : ",num[2])
##data check
        now = time.localtime()
        s = "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year,now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_$
        data1 = s + '/' + data
        chk_data(data1)                 # check data overlap (overlap => chk_flag=1)
        if(chk_flag==1):
                continue                # if data overlap init program
        else:
                pass
##data transmit to MySQL
        on_message(data)                # insert state_GPS_data
        total_time(s,num[2])            # update total_usingtime
        GPS_set_and_distance(num[0],num[1],num[2])      # update total_distance
##option
        sock.sendto(data,addr)
        print(sys.getsizeof(data))
        print(" ")
